In [1]:
import requests
import urllib.request
import json 

import numpy as np

Пример данных

In [2]:
data = (58, 'State-gov',110199,'7th-8th',4,'Married-civ-spouse','Transport-moving','Husband','White','Male',0,0,40,'United-States')

формируем запрос

In [6]:
def send_json(x):
    age, workclass, fnlwgt, education, educational_num,marital_status, occupation, relationship, race, gender,capital_gain, capital_loss, hours_per_week, native_country = x

    body = {'age' : age,
            'workclass' : workclass,
            'fnlwgt' : fnlwgt,
            'education' : education,                               
            'educational-num' : educational_num,
            'marital-status' : marital_status,
            'occupation' : occupation,
            'relationship' : relationship,
            'race' : race,
            'gender' : gender,
            'capital-gain' : capital_gain,
            'capital-loss' : capital_loss,
            'hours-per-week' : hours_per_week,
            'native-country' : native_country}
    
    myurl = 'http://b86a-34-125-211-95.ngrok.io' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [7]:
(age, workclass, fnlwgt, education, educational_num,
       marital_status, occupation, relationship, race, gender,
       capital_gain, capital_loss, hours_per_week, native_country) = (58, 'State-gov',110199, 
                                                                    '7th-8th',4,'Married-civ-spouse','Transport-moving','Husband',
                                                                      'White','Male',0,0,40,'United-States')
body = {'age' : age,
        'workclass' : workclass,
        'fnlwgt' : fnlwgt,
        'education' : education,                               
        'educational-num' : educational_num,
        'marital-status' : marital_status,
        'occupation' : occupation,
        'relationship' : relationship,
        'race' : race,
        'gender' : gender,
        'capital-gain' : capital_gain,
        'capital-loss' : capital_loss,
        'hours-per-week' : hours_per_week,
        'native-country' : native_country} 

обращение к серверу с запросом из одного набора (его построили руками выше - data)

In [8]:
response = send_json(data)
print('предсказание', response)

предсказание 0.14666666666666667


Сделаем обработку массы запросов:

In [9]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("/content/X_test-2.csv")
y_test = pd.read_csv("/content/y_test-2.csv")

In [10]:
X_test[['age','workclass','fnlwgt','education','educational-num','marital-status','occupation','relationship','race','gender',
            'capital-gain',
            'capital-loss',
            'hours-per-week',
            'native-country']].head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,59,Private,109015,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,38,United-States
1,41,Private,289551,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,7688,0,40,United-States
2,23,Private,428299,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States
3,65,Self-emp-inc,103824,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
4,73,Self-emp-not-inc,110102,HS-grad,9,Widowed,Farming-fishing,Not-in-family,White,Male,0,1668,77,United-States


Сделаем N запросов и оценим время

In [17]:
N = 50

In [18]:
%%time
predictions = X_test[
                     ['age','workclass','fnlwgt','education','educational-num','marital-status','occupation','relationship','race','gender',
            'capital-gain',
            'capital-loss',
            'hours-per-week',
            'native-country']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 238 ms, sys: 13.9 ms, total: 252 ms
Wall time: 11.1 s


In [19]:
predictions.values[:5]

array([0.14666667, 0.14666667, 0.14666667, 0.14666667, 0.14666667])

Посчитаем метрику

In [20]:
y_test.head()

,income_>50K
0,1
1,1
2,0
3,1
4,0


In [21]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.14666666666666667, F-Score=0.630, Precision=0.460, Recall=1.000


In [22]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.5